# Making a Mongolian Food Classifier and Deploying It

## Package Installation and Imports

In [ ]:
!pip install -Uqq fastai --upgrade
!pip install -Uqq fastcore

     |████████████████████████████████| 194kB 13.2MB/s 
     |████████████████████▍           | 494.4MB 1.5MB/s eta 0:03:04

In [ ]:
import fastai
import fastcore
print('Fast.ai version:', fastai.__version__)
print('Fastcore version:', fastcore.__version__)

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
!pip install -Uqq azure-cognitiveservices-search-imagesearch
from azure.cognitiveservices.search.imagesearch import ImageSearchClient as api
from msrest.authentication import CognitiveServicesCredentials as auth

In [ ]:
!pip install -Uqq anvil-uplink

## Set Notebook Directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Food Classifier"

In [ ]:
%ls

## Gather Data

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', '161433e0da0b4f168658a54201b384e9')

In [ ]:
def search_images_bing(key, term, min_sz=128, max_images=150):    
     params = {'q':term, 'count':max_images, 'min_height':min_sz, 'min_width':min_sz}
     headers = {"Ocp-Apim-Subscription-Key":key}
     search_url = "https://api.bing.microsoft.com/v7.0/images/search"
     response = requests.get(search_url, headers=headers, params=params)
     response.raise_for_status()
     search_results = response.json()    
     return L(search_results['value'])

In [ ]:
food_types = 'бууз','хуушуур','цуйван'
path = Path('food')

In [ ]:
path

In [ ]:
food_types

In [ ]:
if not path.exists():
    path.mkdir()
    for o in food_types:
        print(f'Finding images for {o}')
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o}')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
??verify_images

## Build Data Loader

In [ ]:
foods = DataBlock(blocks=(ImageBlock, CategoryBlock),
                    get_items=get_image_files,
                    splitter=RandomSplitter(valid_pct=0.2, seed=42),
                    get_y=parent_label,
                    item_tfms=RandomResizedCrop(224, min_scale=0.5), 
                    batch_tfms=aug_transforms())

In [ ]:
dls = food.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=8, nrows=2)

## Perform Transfer Learning

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

### Evaluate Learner

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(15, nrows=3)

## Scrub Images

In [ ]:

cleaner = ImageClassifierCleaner(learn)
cleaner

## Export Model

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

## Anvil Function

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
import anvil.server
import anvil.media

anvil.server.connect("LVUI6C7UDE54Q62RZC2YK3FW-EDNORO2NSVZEF6Y3")

In [ ]:
@anvil.server.callable
def classify_image(file):
    with anvil.media.TempFile(file) as f:
        img = PILImage.create(f)
    
    pred, pred_idx, probs = learn_inf.predict(img)

    return pred, max(probs.tolist())

In [ ]:
%ls